## Automatic Extraction of Gravity Waves from All-Sky Airglow Image Based on Machine Learning, 
#### by Chang Lai 1,2,*,Jiyao Xu 2,Jia Yue 3ORCID,Wei Yuan 2,Xiao Liu 4,Wei Li 1 andQinzeng Li 2

### A CNN is deployed addressing the archtecure of the above paper
#### *Seraj Mostafa, Dept. of IS, UMBC, MD, USA*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import keras
import os

In [ ]:
import tensorflow
from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_path = '/path/to-dir''
os.chdir(data_path)
print(os.path.abspath(os.getcwd()))

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(.5))

model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(.5))

model.add(layers.Flatten())
model.add(layers.Dropout(.5))

model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
batch_size = 64

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

train_list = []
batch_index = 0
while batch_index <= train_generator.batch_index:
  train = train_generator.next()
  train_list.append(train[0])
  batch_index = batch_index + 1
train_array = np.asarray(train_list)

validation_generator = test_datagen.flow_from_directory( 
        './validation',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

validation_list = []
batch_index = 0
while batch_index <= validation_generator.batch_index:
  validation = validation_generator.next()
  validation_list.append(validation[0])
  batch_index = batch_index + 1
validation_array = np.asarray(validation_list)

test_generator = test_datagen.flow_from_directory(
        './test',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary',
        )

test_list = []
batch_index = 0
while batch_index <= test_generator.batch_index:
  test = test_generator.next()
  test_list.append(test[0])
  batch_index = batch_index + 1 
test_array = np.asarray(test_list)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
h= model.fit(
        train_generator,
        epochs=100,
        validation_data=validation_generator,
        steps_per_epoch=200 // batch_size,
        validation_steps=80 // batch_size
        )

In [ ]:
model.evaluate(train_generator)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
model.evaluate(test_generator)

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

for batch in validation:
  X,y = validation
  yhat = model.predict(X)
  pre.update_state(y, yhat)
  re.update_state(y, yhat)
  acc.update_state(y, yhat)

f1_score = (2 * pre.result().numpy() * re.result())/(pre.result().numpy() + re.result())

print('For Validation: ')
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}, F1 Score: {f1_score}')

In [ ]:
from sklearn import metrics

x = model.predict(train_generator)
metrics.roc_auc_score(train_generator.labels, x)

In [ ]:
model.save('/content/Lai.et.al.CNN.h5')
print('Model Saved!')